# Multiple Linear Regression

## 선형 가정
1. 직선성(linear) 변수간의 선형성이 있어야.
2. Homoscedasticity : 등분산
3. multivariate normality: Normality of error distribution
4. Independence
5. Lack of multicollinearity : 다중공선성 없음.
6. Outlier check : 이상치 쓸건지 말건지 판단해야

## Building a Model
1. All in : 다넣기
2. Backward Elimination : all in에서 다 넣은 거보고 p-value 높은거 같다 버림. 그상태로 다시 모델 fit 반복
    p값이 유의수준보다 다 낮을때까지 해보자.
3. Forward selection : 반대임. simple regression부터 시작해서 하나하나 넣음. 딱봐도 오래걸리쥬?
4. Bidirectional Elimination : 순방향 기반으로 하고 후진 제거 이걸 반복
5. All possible model : feature 조합별로 모든 모델을 봄. best performance 나오는거 고르기.

## Scaling?
다중 선형회귀는 각 변수마다 계수를 가지고 있으니까 알아서 계수로 scaling이 됨.
그러니까 따로 Scaling 안해줘도댐

## Importing the libraries

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Importing the dataset

In [62]:
data =  pd.read_csv('50_Startups.csv')
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [63]:
print(X,y)

[[165349.2 136897.8 471784.1 'New York']
 [162597.7 151377.59 443898.53 'California']
 [153441.51 101145.55 407934.54 'Florida']
 [144372.41 118671.85 383199.62 'New York']
 [142107.34 91391.77 366168.42 'Florida']
 [131876.9 99814.71 362861.36 'New York']
 [134615.46 147198.87 127716.82 'California']
 [130298.13 145530.06 323876.68 'Florida']
 [120542.52 148718.95 311613.29 'New York']
 [123334.88 108679.17 304981.62 'California']
 [101913.08 110594.11 229160.95 'Florida']
 [100671.96 91790.61 249744.55 'California']
 [93863.75 127320.38 249839.44 'Florida']
 [91992.39 135495.07 252664.93 'California']
 [119943.24 156547.42 256512.92 'Florida']
 [114523.61 122616.84 261776.23 'New York']
 [78013.11 121597.55 264346.06 'California']
 [94657.16 145077.58 282574.31 'New York']
 [91749.16 114175.79 294919.57 'Florida']
 [86419.7 153514.11 0.0 'New York']
 [76253.86 113867.3 298664.47 'California']
 [78389.47 153773.43 299737.29 'New York']
 [73994.56 122782.75 303319.26 'Florida']
 [67532

자 데이터를 봤을 떄 State 가 3개로 되어있는데 2개일 때를 생각해보자 ㄱ,ㄴ State라고 하자.

다중 선형회귀를 할 때 보통 ㄱ과 ㄴ을 원핫인코딩을 한다. 
그럼 더미데이터가 2열이 된다. ㄱ열과 ㄴ열. 근데 이미 ㄱ열에 1 or 0이니까 사실 ㄴ열이 필요없다. 빼고 한다면 다중 선형회귀식에 ㄱ에 대한 계수만 나올것이다.

자 문제는 그럼 ㄱ일때 계수는 있지만 ㄴ일때 계수는 존재하지 않다.

근데 상관없다 왜냐면 **ㄴ일 경우의 계수가 이미 상수항에 포함되어 있는 것이다**

그러니까 State가 1일 경우 ㄱ의 식이 되는거고 0일 경우 ㄴ의 식이 된다는 것.

만약에 ㄱ항과 ㄴ항을 넣어주면 어떻게 될까. 이건 문제가 생긴다.

ㄱ의 x1과 ㄴ의 x2라고 할 때, x1 = 1-x2가 된다. 자 이러면 **다중공선성**의 문제가 생긴다.

그러니까 결론은?? **더미 변수가 3개다.. 그럼 2개만 넣어주고, 10개다 하면 9개만 넣어주자.**

## Encoding categorical data

In [70]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ohe = ColumnTransformer(transformers=[('encode', OneHotEncoder(), [3])], remainder='passthrough')
X = np.array(ohe.fit_transform(X))

In [71]:
X

array([[0.0, 0.0, 1.0, 165349.2, 136897.8, 471784.1],
       [1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [0.0, 1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [0.0, 1.0, 0.0, 142107.34, 91391.77, 366168.42],
       [0.0, 0.0, 1.0, 131876.9, 99814.71, 362861.36],
       [1.0, 0.0, 0.0, 134615.46, 147198.87, 127716.82],
       [0.0, 1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 0.0, 1.0, 120542.52, 148718.95, 311613.29],
       [1.0, 0.0, 0.0, 123334.88, 108679.17, 304981.62],
       [0.0, 1.0, 0.0, 101913.08, 110594.11, 229160.95],
       [1.0, 0.0, 0.0, 100671.96, 91790.61, 249744.55],
       [0.0, 1.0, 0.0, 93863.75, 127320.38, 249839.44],
       [1.0, 0.0, 0.0, 91992.39, 135495.07, 252664.93],
       [0.0, 1.0, 0.0, 119943.24, 156547.42, 256512.92],
       [0.0, 0.0, 1.0, 114523.61, 122616.84, 261776.23],
       [1.0, 0.0, 0.0, 78013.11, 121597.55, 264346.06],
       [0.0, 0.0, 1.0, 94657.16, 145077.58

## Splitting the dataset into the Training set and Test set

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

## Training the Multiple Linear Regression model on the Training set
1. 앞서 말한 더미데이터의 다중공선성을 치하기 위해 뭔가를 해야하나?
> 아니다. 왜냐하면 다중 선형회귀 class는 다중 선형회귀 모델을 빌드하고 훈련도하는 class임
> 어렵게 말했는데 대충.. 알아서 피해줌. 알아서 더미변수중 하나 빼줌.

2. 역방향, 순방향등 해야하나?
> 필요없음 알아서 해줄거임.

# Training the Multiple Linear Regression model on the Training set

In [74]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

## Predicting the Test set results

In [79]:
y_pred = model.predict(X_test)

# 아래 코드로 소수접 이하의 출력 형식 조정. 소수점 2까지
np.set_printoptions(precision=2)

# 각 값을 n행 1열로 열벡터화 시켜줌. np.cconcatenate(axis=1)로 수평으로 결합.
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),axis=1))

[[126362.88 134307.35]
 [ 84608.45  81005.76]
 [ 99677.49  99937.59]
 [ 46357.46  64926.08]
 [128750.48 125370.37]
 [ 50912.42  35673.41]
 [109741.35 105733.54]
 [100643.24 107404.34]
 [ 97599.28  97427.84]
 [113097.43 122776.86]]


In [84]:
# 원하는 값을 넣어서 한번 예측값을 보자
# Califonia, R&D spend=140000, Administration Spend=125300, Marketing spend=340000
print(model.predict([[1,0,0,140000,125300,340000]]))

[168347.98]


In [ ]:
# 계수 값 구하기
print(